# Load jdaviz app in Mosviz configuration

In [ ]:
from jdaviz.configs.mosviz.helper import Mosviz

In [ ]:
mosviz = Mosviz()
mosviz.show()

## Load data

In [ ]:
import glob

spectra_1d = []
spectra_2d = []
images = []

from astropy.utils.data import download_file
import tempfile
import pathlib
from zipfile import ZipFile

fn = download_file('https://stsci.box.com/shared/static/ovyxi5eund92yoadvv01mynwt8t5n7jv.zip', cache=True)
with ZipFile(fn, 'r') as sample_data_zip:
    # Unzip sample data archive
    sample_data_zip.extractall(tempfile.gettempdir())
    
level3_path = (pathlib.Path(tempfile.gettempdir()) / 'mosviz_nirspec_data_0.3' / 'level3')
for file_path in glob.glob(str(level3_path / '*')):
    if 'x1d' in file_path:
        spectra_1d.append(file_path)
    elif 's2d' in file_path:
        spectra_2d.append(file_path)
        
        
for i in range(len(spectra_1d)):
    images.append(str(level3_path / 'mosviz_cutouts' / '227.fits'))

mosviz.load_1d_spectra(spectra_1d)
mosviz.load_2d_spectra(spectra_2d)
mosviz.load_images(images)

# Everything below this is test code used for development
---

In [ ]:
fig_image = mosviz.app.get_viewer("image-viewer").figure

In [ ]:
import bqplot
from regions import RectanglePixelRegion, PixCoord
from ipywidgets import VBox
import numpy as np
from bqplot import *

def region_to_vertices(region):
    p = region.to_polygon()
    return p.vertices.x, p.vertices.y

# Extract slit data from files and visualize

In [ ]:
# Load 2d spectrum data on the right image viewer
#spectrum_2d_file = "/mosviz_nirspec_data_0.3/level3/f170lp-g235m_mos_observation-6-c0e0_s00227_cal.fits"
# app.load_data(spectrum_2d_file)

In [ ]:
#spectrum_2d_file = "/mosviz_nirspec_data_0.3/level3/f170lp-g235m_mos_observation-6-c0e0_s00482_s2d.fits"
# from astropy.io import fits
# hdu = fits.open(spectrum_2d_file)
# header = hdu[1].header
# [print(x) for x in header]

### Set 2D Spectrum viewer (top right) to visualize f170lp-g235m_mos_observation-6-c0e0_s00227_cal.fits

In [ ]:
fig_2d = mosviz.app.get_viewer("spectrum-2d-viewer").figure

In [ ]:
from astropy.io import fits
hdu = fits.open(spectrum_2d_file)
header = hdu[1].header
[print(x) for x in header]

In [ ]:
# Code to create SkyRegions

from regions import  RectangleSkyRegion,RectanglePixelRegion
from astropy.coordinates import Angle, SkyCoord
from astropy import units as u 
from astropy.wcs import WCS
def jwst_header_to_skyregion(header):
    s_region = header['S_REGION']
    footprint = s_region.split("POLYGON ICRS")[1].split()
    ra = np.array(footprint[::2], dtype=np.float)
    dec = np.array(footprint[1::2], dtype=np.float)
        
    # Find center of slit
    cra = (max(ra) + min(ra))/2 
    cdec = (max(dec) + min(dec))/2
    
    # Find center as skycoord
    skycoord = SkyCoord(cra, cdec,
                        unit=(u.Unit(u.deg),
                              u.Unit(u.deg)))
    
    # Puts corners of slit into skycoord object
    corners = SkyCoord(ra, dec, unit="deg")
    
    # Compute length and width
    length = corners[0].separation(corners[1])
    width = corners[1].separation(corners[2])
    length = Angle(length, u.deg)
    width = Angle(width, u.deg)
    
    skyregion = RectangleSkyRegion(center=skycoord, width=width, height=length)
    return skyregion

In [ ]:
# Set WCS

sky_region = jwst_header_to_skyregion(header)
header_copy = header.copy()
header_copy['WCSAXES'] = 2
for key in header:
    if '3' in key:
        header_copy.remove(key)
wcs_2d = WCS(header_copy)


In [ ]:
# Use wcs of image viewer to scale slit dimensions correctly
image_hdu = fits.open(cutout_image_file)
wcs_image = WCS(image_hdu[0].header)

pixel_region = sky_region.to_pixel(wcs_image)


In [ ]:
# Create polygon region from the pixel region and set vertices
pix_rec = pixel_region.to_polygon()

x_coords = pix_rec.vertices.x
y_coords = pix_rec.vertices.y

# Create LinearScale that is the same size as the image viewer
scales={'x': fig_image.interaction.x_scale, 'y':fig_image.interaction.y_scale}

# Create slit
patch2 = bqplot.Lines(x=x_coords, y=y_coords, scales=scales, fill='none', colors = ["red"], stroke_width=2, close_path=True)

# Visualize slit on the figure
fig_image.marks = fig_image.marks + [patch2]
fig_image

print(x_coords, y_coords, scales)

In [ ]:
fig_image = mosviz.app.get_viewer("image-viewer").figure
fig_image.axes[1].tick_format = None
fig_image.axes[0].tick_format = None

fig_image.axes[1].label = "y: pixels"
fig_image.axes[0].label = "x: pixels"

In [ ]:
table = mosviz.app.get_viewer('table-viewer')

In [ ]:
dir(table)

In [ ]:
table._on_row_selected()

In [ ]:
spec1d = mosviz.app.get_viewer("spectrum-viewer")
spec1d.figure.axes[1].tick_format = "font_size: 6"
spec1d.figure.axes[1]

In [ ]:
spec2d = mosviz.app.get_viewer("spectrum-2d-viewer")
spec2d.figure.axes[0].visible = False
spec2d.figure.axes[1].label = "y: pixel"
spec2d.figure.axes[1].tick_format = None
spec2d.figure.axes[1].label_location = 'start'

spec2d.figure.min_aspect_ratio

In [ ]:
data = mosviz.app.get_viewer("image-viewer").data()
data[0].get_object().meta

In [ ]:
spec2d_data = spec2d.data()[0]
spec2d_data.wcs

In [ ]:
spec1d_data = spec1d.data()[0]
spec1d_data.unit